In [1]:
# Import Libraries

# System

import os

# Analysis

import numpy as np # Numerical Analysis
import arcpy 

cwd = r"C:\\Users\\HAL\\Desktop\\arc1_lab3_part1\\"
os.chdir(cwd)
arcpy.env.workspace = os.path.join(cwd, "Arc1_Lab3_Part1.gdb")

In [2]:
# Get Start and end coordinates

# Load Dory's Trip

arcpy.conversion.JSONToFeatures("DoryTrip.geojson",
                                "DoryTrip",
                                "POINT")

# Select end and start

arcpy.analysis.Select("DoryTrip",
                      "DoryTrip_end",
                      "type = 'end'")

arcpy.analysis.Select("DoryTrip",
                      "DoryTrip_start",
                      "type = 'start'")

<Result 'C:\\\\Users\\\\HAL\\\\Desktop\\\\arc1_lab3_part1\\\\Arc1_Lab3_Part1.gdb\\DoryTrip_start'>

In [3]:
# Get all tifnames

file_names = np.array(os.listdir('5_weights_tests'))


In [4]:
# Get least cost paths

paths = []

for i, filename in enumerate(file_names):
    
    if filename[0] != '0': # We don't want any of the costpaths without slope (doesn't work)
    
        weightnames = filename[:-17] # Weights name (layer name without '-Cost_Surface.tif')

        # Import cost surface

        cs_path = os.path.join(os.getcwd(), '5_weights_tests', filename)
        in_raster = arcpy.Raster(cs_path)

        # Get Path Distance and backlink rasters
        out_distance_raster = arcpy.sa.CostDistance("DoryTrip_start", in_cost_raster = in_raster, 
                                                    out_backlink_raster = "PathdistBacklink");
        out_distance_raster.save("Pathdist")
        
        # Get costpath
        
        costpath_savename = os.path.join(os.getcwd(), '6_costpaths', weightnames + '_CostPath.tif')

        least_cost_path = arcpy.sa.CostPath("DoryTrip_end",
                                       'Pathdist', 
                                       "PathdistBacklink", 
                                       "BEST_SINGLE");least_cost_path.save(costpath_savename)
        
        # Make sure these aren't used again by accident
        arcpy.Delete_management("Pathdist")
        arcpy.Delete_management("PathdistBacklink")